# redun's scheduler walk through

In this notebook, we'll walk through how redun's scheduler implements workflow execution. Briefly, redun represents workflows as computional expressions that are iteratively evaluated by a scheduler using a technique called [graph reduction](https://en.wikipedia.org/wiki/Graph_reduction). This approach allows users to specify large computational workflows in a very natural and expressive style.

If you haven't already please see the first two examples for an introduction to redun's basic features. Those examples executed workflows using the `redun` CLI (command-line interface), which is the most common way to execute a workflow. In this notebook, we will run workflows programmatically using the `Scheduler` class in order to inspect the execution process more closely. The redun CLI is just a thin wrapper around the `Scheduler`.

In [1]:
# Imports.
import logging, time

from redun import Scheduler, task

In [2]:
# Create a minimal Scheduler that uses in-memory cache.

scheduler = Scheduler()
scheduler.logger.setLevel(logging.INFO)

[redun] Upgrading db from version -1.0 to 3.1...


In [3]:
# Create a simple redun task.

# Tasks will automatically use the global variable `redun_namespace` as their namespace.
redun_namespace = "redun_notebook"

@task()
def add(a, b):
    return a + b

In [4]:
# The `@task` decorator turns functions into Task objects.
add

Task(fullname=redun_notebook.add, hash=496bc317)

In [5]:
# Tasks have several properties, such as a name, a hash, their source code, and the original wrapped function.
print("name", add.name)
print("hash", add.hash)
print("func", add.func)
print(add.source)

name add
hash 496bc31711bb263aa03d3064f2c6939d8cccec42
func <function add at 0x7ff1b0e3a310>
def add(a, b):
    return a + b



In [6]:
# The key trick is that task calls are intercepted and return Expressions.
add(1, 2)

redun_notebook.add(1, 2)

In [7]:
# The most common Expression is a TaskExpression. The following Expression,
#   redun_notebook.add(1, 2)
# represents calling the task 'redun_notebook.add' with positional arguments (1, 2).

In [8]:
# To evaluate an expression, use scheduler.run():
scheduler.run(add(1, 2))

[redun] Start Execution 7aed5426-cf0f-427f-9d63-116e227fb48a:  redun 'redun_notebook.add(1, 2)'
[redun] Run    Job 7dd95924:  redun_notebook.add(a=1, b=2) on default
[redun] 
[redun] | JOB STATUS 2022/09/22 04:57:11
[redun] | TASK               PENDING RUNNING  FAILED  CACHED    DONE   TOTAL
[redun] | 
[redun] | ALL                      0       0       0       0       1       1
[redun] | redun_notebook.add       0       0       0       0       1       1
[redun] 
[redun] 
[redun] Execution duration: 0.03 seconds


3

In [9]:
# Expressions can be combined into an expression tree/graph.
add(add(1, 2), 3)

redun_notebook.add(redun_notebook.add(1, 2), 3)

In [10]:
# We can evaluate a whole expression tree using the scheduler:
scheduler.run(add(add(1, 2), 3))

[redun] Start Execution a4b659bd-70c5-4554-b805-939448b89301:  redun 'redun.root_task(QuotedExpression(redun_notebook.add(redun_notebook.add(1, 2), 3)))'
[redun] Run    Job 019dc9e4:  redun.root_task(expr=QuotedExpression(redun_notebook.add(redun_notebook.add(1, 2), 3))) on default
[redun] Cached Job 6fd11ac6:  redun_notebook.add(a=1, b=2) (eval_hash=a162a83f)
[redun] Run    Job 9034dca3:  redun_notebook.add(a=3, b=3) on default
[redun] 
[redun] | JOB STATUS 2022/09/22 04:57:11
[redun] | TASK               PENDING RUNNING  FAILED  CACHED    DONE   TOTAL
[redun] | 
[redun] | ALL                      0       0       0       1       2       3
[redun] | redun.root_task          0       0       0       0       1       1
[redun] | redun_notebook.add       0       0       0       1       1       2
[redun] 
[redun] 
[redun] Execution duration: 0.05 seconds


6

In [11]:
# We have defined a small workflow/pipeline using what others call "invocation style".

In [12]:
# Let's redefine the add() task to be artificially slower, so we can more easily see tasks running in parallel.

@task(version='1')
def add(a, b):
    print(f"start add({a}, {b})")
    time.sleep(1)
    print(f"stop  add({a}, {b})")
    return a + b

In [13]:
# Task source and hash should now be updated.

print(add.source)

add.hash

def add(a, b):
    print(f"start add({a}, {b})")
    time.sleep(1)
    print(f"stop  add({a}, {b})")
    return a + b



'037d0e72cc62a8360d7cb321a68135eb790f17cd'

In [14]:
# Expressions that do not depend on each other (e.g. `add(1, 2)` and `add(3, 4)`) will 
# automatically run in parallel. Notice how we see two starts and then two stops.
scheduler = Scheduler()
scheduler.run(add(add(1, 2), add(3, 4)))

[redun] Upgrading db from version -1.0 to 3.1...
[redun] Start Execution bcf4dcac-4ac1-4b34-98c3-ce7f3d0c9b10:  redun 'redun.root_task(QuotedExpression(redun_notebook.add(redun_notebook.add(1, 2), redun_notebook.add(3, 4))))'
[redun] Run    Job 2a1339e2:  redun.root_task(expr=QuotedExpression(redun_notebook.add(redun_notebook.add(1, 2), redun_notebook.add(3, 4)))) on default
[redun] Run    Job 46fdcfd3:  redun_notebook.add(a=1, b=2) on default
[redun] Run    Job 6b57f1f8:  redun_notebook.add(a=3, b=4) on default


start add(1, 2)
start add(3, 4)


[redun] Run    Job de859690:  redun_notebook.add(a=3, b=7) on default


stop  add(1, 2)
stop  add(3, 4)
start add(3, 7)


[redun] 
[redun] | JOB STATUS 2022/09/22 04:57:14
[redun] | TASK               PENDING RUNNING  FAILED  CACHED    DONE   TOTAL
[redun] | 
[redun] | ALL                      0       0       0       0       4       4
[redun] | redun.root_task          0       0       0       0       1       1
[redun] | redun_notebook.add       0       0       0       0       3       3
[redun] 
[redun] 
[redun] Execution duration: 2.08 seconds


stop  add(3, 7)


10

## Executing workflows with graph reduction

The redun scheduler treats the expression `add(add(1, 2), add(3, 4))` as a tree (or DAG more generally). In the figure below, concrete values (anything that's not a subclass of `Expression`) are drawn in grey boxes, and `TaskExpressions` are drawn in blue boxes.

<img width="50%" src="images/expression-tree.png">

When `scheduler.run()` is applied, it begins to identify subtrees that can be *reduced*, namely tasks with concrete arguments (see red boxes below). The scheduler will perform these reductions by calling tasks (e.g. `add`) with the child nodes as the arguments. These task executions can occur in parallel, which is safe to do because tasks are required to be written as pure functions with no side-effects. The scheduler can also consult the cache before performing each reduction to see if it can skip executing a task and instead just replay the result.

<img width="50%" src="images/expression-tree2.png">

When a task completes its execution, its result is replaced into the expression tree.

<img width="25%" src="images/expression-tree3.png">

As reductions complete, additional reductions become possible.

<img width="25%" src="images/expression-tree4.png">

Eventually the expression graph reduces to a single concrete value and the workflow execution concludes.

<img width="12%" src="images/expression-tree5.png">

In [15]:
# Let's say we add four numbers a lot. Let's abstract it into its own task.

@task()
def add4(a, b, c, d):
    return add(add(a, b), add(c, d))

In [16]:
scheduler.run(add4(5, 6, 7, 8))

[redun] Start Execution 538b1da6-b79d-4102-81c4-e92720516740:  redun 'redun_notebook.add4(5, 6, 7, 8)'
[redun] Run    Job 7863dc2a:  redun_notebook.add4(a=5, b=6, c=7, d=8) on default
[redun] Run    Job 778b37a6:  redun_notebook.add(a=5, b=6) on default
[redun] Run    Job 6c5bab49:  redun_notebook.add(a=7, b=8) on default


start add(5, 6)
start add(7, 8)


[redun] Run    Job c07ba7d4:  redun_notebook.add(a=11, b=15) on default


stop  add(5, 6)stop  add(7, 8)

start add(11, 15)


[redun] 
[redun] | JOB STATUS 2022/09/22 04:57:16
[redun] | TASK                PENDING RUNNING  FAILED  CACHED    DONE   TOTAL
[redun] | 
[redun] | ALL                       0       0       0       0       4       4
[redun] | redun_notebook.add        0       0       0       0       3       3
[redun] | redun_notebook.add4       0       0       0       0       1       1
[redun] 
[redun] 
[redun] Execution duration: 2.08 seconds


stop  add(11, 15)


26

In [17]:
# This returns the expected result. If a redun task returns an expression, 
# the scheduler will perform a follow up evaluation.

## Graph reductions that return more expressions

The expression `add4(5, 6, 7, 8)` would look like the following expression tree.

<img width="50%" src="images/expression-tree6.png">

As we saw above, instead of returning a concrete value, task `add4` actually returned more expressions. This is a completely valid thing to do, and when it happens, the redun scheduler will simply place those expressions into a expression graph.

<img width="50%" src="images/expression-tree7.png">

The scheduler will begin to recurse into the new expressions looking for more reductions. 

In this way, the expression graph can actually expand and contract. Being able to dynamically grow the expression graph is what gives redun its ability to express very dynamic workflows.

This process actually follows the implementation of classic functional programming language compilers and interpreters.

In [18]:
# Let's generalize 'add' to arbitrary number of values.

@task()
def add_many(values, init=0):
    if len(values) == 0:
        return init
    elif len(values) == 1:
        return values[0]
    else:
        k = len(values) // 2
        sum1 = add_many(values[:k])
        sum2 = add_many(values[k:])
        return add(sum1, sum2)

In [19]:
# As this expression is evaluated imagine how the expression tree is expanding and contracting.
scheduler.run(add_many(list(range(100))))

[redun] Start Execution a5666657-bdcc-45a7-9760-827fa11d5790:  redun 'redun_notebook.add_many([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45...'
[redun] Run    Job 6662a0ef:  redun_notebook.add_many(values=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51..., init=0) on default
[redun] Run    Job e74a1beb:  redun_notebook.add_many(values=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49], init=0) on default
[redun] Run    Job 0dcbebf2:  redun_notebook.add_many(values=[50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 

[redun] Run    Job ab7a8823:  redun_notebook.add_many(values=[12], init=0) on default
[redun] Run    Job 15e26e99:  redun_notebook.add_many(values=[13, 14], init=0) on default
[redun] Run    Job c4a422ef:  redun_notebook.add_many(values=[15], init=0) on default
[redun] Run    Job c87c0e8e:  redun_notebook.add_many(values=[16, 17], init=0) on default
[redun] Run    Job 873b5ba9:  redun_notebook.add_many(values=[18], init=0) on default
[redun] Run    Job 5014dae6:  redun_notebook.add_many(values=[19, 20], init=0) on default
[redun] Run    Job 26ff76cc:  redun_notebook.add_many(values=[21, 22], init=0) on default
[redun] Run    Job 65f0c4a3:  redun_notebook.add_many(values=[23, 24], init=0) on default
[redun] Run    Job 15360388:  redun_notebook.add_many(values=[25], init=0) on default
[redun] Run    Job 7ec36d7e:  redun_notebook.add_many(values=[26, 27], init=0) on default
[redun] Run    Job aef98ac7:  redun_notebook.add_many(values=[28], init=0) on default
[redun] Run    Job 02b17256:  

[redun] Run    Job d4f68eca:  redun_notebook.add_many(values=[54], init=0) on default
[redun] Run    Job f5191c8f:  redun_notebook.add_many(values=[55], init=0) on default
[redun] Run    Job 4c1cb156:  redun_notebook.add_many(values=[57], init=0) on default
[redun] Run    Job d4e902e5:  redun_notebook.add_many(values=[58], init=0) on default
[redun] Run    Job b59e6c05:  redun_notebook.add_many(values=[60], init=0) on default
[redun] Run    Job 82c09638:  redun_notebook.add_many(values=[61], init=0) on default
[redun] Run    Job 3c6e0a1b:  redun_notebook.add_many(values=[63], init=0) on default
[redun] Run    Job a162a9b1:  redun_notebook.add_many(values=[64], init=0) on default
[redun] Run    Job e19909c2:  redun_notebook.add_many(values=[66], init=0) on default
[redun] Run    Job 3f247a0d:  redun_notebook.add_many(values=[67], init=0) on default
[redun] Run    Job d8eb4998:  redun_notebook.add_many(values=[69], init=0) on default
[redun] Run    Job ebc52427:  redun_notebook.add_many(

start add(4, 5)
start add(10, 11)
start add(13, 14)
start add(16, 17)
start add(19, 20)
start add(21, 22)
start add(23, 24)
start add(26, 27)
start add(29, 30)
start add(32, 33)
start add(35, 36)
start add(38, 39)
start add(41, 42)
start add(44, 45)
start add(46, 47)
start add(48, 49)
start add(51, 52)
start add(54, 55)
start add(57, 58)
start add(60, 61)


[redun] Run    Job abdde0ef:  redun_notebook.add(a=3, b=9) on default
[redun] Run    Job 23a760e3:  redun_notebook.add(a=9, b=21) on default
[redun] Run    Job 7cf39161:  redun_notebook.add(a=12, b=27) on default
[redun] Run    Job 9e8111d4:  redun_notebook.add(a=15, b=33) on default


...

[redun] Run    Job 479bf2cc:  redun_notebook.add(a=1225, b=3725) on default


stop  add(1550, 2175)
start add(1225, 3725)


[redun] 
[redun] | JOB STATUS 2022/09/22 04:57:27
[redun] | TASK                    PENDING RUNNING  FAILED  CACHED    DONE   TOTAL
[redun] | 
[redun] | ALL                           0       0       0       2     296     298
[redun] | redun_notebook.add            0       0       0       2      97      99
[redun] | redun_notebook.add_many       0       0       0       0     199     199
[redun] 
[redun] 
[redun] Execution duration: 11.03 seconds


stop  add(1225, 3725)


4950

In [20]:
# Let's suppress the scheduler's logging output.

import logging
scheduler.logger.setLevel(logging.ERROR)

In [21]:
# Let's try a scatter-gather pipeline.

# Scatter (aka map).
init_values = list(range(10))
values = [add(i, 1) for i in init_values]

# Gather (aka reduce).
total = add_many(values)
scheduler.run(total)

start add(0, 1)
start add(1, 1)
start add(2, 1)
start add(3, 1)
start add(4, 1)
start add(5, 1)
start add(6, 1)
start add(7, 1)
start add(8, 1)
start add(9, 1)
stop  add(0, 1)stop  add(1, 1)
stop  add(2, 1)

stop  add(3, 1)
stop  add(4, 1)
stop  add(5, 1)
stop  add(6, 1)
stop  add(7, 1)
stop  add(8, 1)
stop  add(9, 1)
start add(6, 7)
start add(9, 10)
stop  add(6, 7)
stop  add(9, 10)
start add(8, 19)
stop  add(8, 19)
start add(13, 27)
stop  add(13, 27)
start add(15, 40)
stop  add(15, 40)


55

In [22]:
# Notice `values` was actually a list of Expressions.
values

[redun_notebook.add(0, 1),
 redun_notebook.add(1, 1),
 redun_notebook.add(2, 1),
 redun_notebook.add(3, 1),
 redun_notebook.add(4, 1),
 redun_notebook.add(5, 1),
 redun_notebook.add(6, 1),
 redun_notebook.add(7, 1),
 redun_notebook.add(8, 1),
 redun_notebook.add(9, 1)]

redun's primary evaluation rule is:

```py
eval(TaskExpression(func, arg1, arg2, ...)) => eval(func(eval(arg1), eval(arg2), ...))
```

In addition, the redun scheduler recurses into "nested values", looking for expressions to evaluate.
That is achieved using additional evaluation rules:

```py
# Evaluate list
eval([x for x in xs]) => [eval(x) for x in xs]

# Evaluate dict
eval({k: v for k, v in dct.items()}) => {eval(k): eval(v) for k, v in dct.items()}

# Evaluate set
eval({x for x in xs}) => {eval(x) for x in xs}

# Evaluate tuple
eval(tuple(x for x in xs)) => tuple(eval(x) for x in xs)

# Evaluate namedtuple
eval(MyNamedTuple(x for x in xs)) => MyNamedTuple(eval(x) for x in xs)
```

In [23]:
# Let's tryout some of these evaluation rules.

In [24]:
expr = {"first": add(1, 2), "second": add(3, 4)}
print(expr)
scheduler.run(expr)

{'first': redun_notebook.add(1, 2), 'second': redun_notebook.add(3, 4)}


{'first': 3, 'second': 7}

In [25]:
expr = [
    {
        "id": 1,
        "price": add(1, 2),
    },
    {
        "id": 2,
        "price": add(3, 4),
    }
]
print(expr)
scheduler.run(expr)

[{'id': 1, 'price': redun_notebook.add(1, 2)}, {'id': 2, 'price': redun_notebook.add(3, 4)}]


[{'id': 1, 'price': 3}, {'id': 2, 'price': 7}]

In [26]:
# Elements of a list do not need to be independent.
# For example, here we are doing a cumulative summation where each entry depends on the previous.

values = list(range(10))
cumsum = [0]
for value in values:
    cumsum.append(add(cumsum[-1], value))

print(cumsum)
scheduler.run(cumsum)

[0, redun_notebook.add(0, 0), redun_notebook.add(redun_notebook.add(0, 0), 1), redun_notebook.add(redun_notebook.add(redun_notebook.add(0, 0), 1), 2), redun_notebook.add(redun_notebook.add(redun_notebook.add(redun_notebook.add(0, 0), 1), 2), 3), redun_notebook.add(redun_notebook.add(redun_notebook.add(redun_notebook.add(redun_notebook.add(0, 0), 1), 2), 3), 4), redun_notebook.add(redun_notebook.add(redun_notebook.add(redun_notebook.add(redun_notebook.add(redun_notebook.add(0, 0), 1), 2), 3), 4), 5), redun_notebook.add(redun_notebook.add(redun_notebook.add(redun_notebook.add(redun_notebook.add(redun_notebook.add(redun_notebook.add(0, 0), 1), 2), 3), 4), 5), 6), redun_notebook.add(redun_notebook.add(redun_notebook.add(redun_notebook.add(redun_notebook.add(redun_notebook.add(redun_notebook.add(redun_notebook.add(0, 0), 1), 2), 3), 4), 5), 6), 7), redun_notebook.add(redun_notebook.add(redun_notebook.add(redun_notebook.add(redun_notebook.add(redun_notebook.add(redun_notebook.add(redun_noteb

[0, 0, 1, 3, 6, 10, 15, 21, 28, 36, 45]

In [27]:
# Let's implement a task with two outputs. Just use a container type, such as dict, list, namedtuple, etc.
# redun has no need for special syntax for multiple outputs.

@task()
def intdiv(numerator, denominator):
    quotient, remainder = divmod(numerator, denominator)
    return {
        "quotient": quotient,
        "remainder": remainder,
    }

In [28]:
# Key and attribute access can be done lazily.
# This allows us to route just some of the output to another task.
intdiv(12, 5)["quotient"]

redun_notebook.intdiv(12, 5)['quotient']

In [29]:
# Here we route the two outputs to different downstream computations.
result = intdiv(12, 5)
x = add(result["quotient"], 1)
y = add(result["remainder"], 2)
scheduler.run(add(x, y))

start add(2, 2)
stop  add(2, 2)


7

In [30]:
class User:
    def __init__(self, name, friend=None):
        self.name = name
        self.friend = friend
    
    def get_friend(self):
        return self.friend

@task()
def returns_alot():
    return User("Alice", friend=User("Bob"))

# Many operations can be chained lazily. For example, attribute access and function calls.
expr = returns_alot().get_friend().name
expr

redun_notebook.returns_alot().get_friend().name

In [31]:
scheduler.run(expr)

'Bob'

In [32]:
# Most operators can be used lazily.
expr = (add(1, 2) + 1) * add(1, 1)
print(expr)
scheduler.run(expr)

((redun_notebook.add(1, 2) + 1) * redun_notebook.add(1, 1))


8

In [33]:
# Tasks are first-class values and can be used as arguments and return values.
# Let's create a higher-order task that performs divide and conquer generically.

@task()
def divide_and_conquer(a_task, values, init=0):
    if len(values) == 0:
        return init
    elif len(values) == 1:
        return values[0]
    else:
        k = len(values) // 2
        sum1 = divide_and_conquer(a_task, values[:k])
        sum2 = divide_and_conquer(a_task, values[k:])
        return a_task(sum1, sum2)

In [34]:
scheduler.run(divide_and_conquer(add, list(range(10))))

start add(8, 9)
start add(2, 7)
stop  add(8, 9)
stop  add(2, 7)
start add(7, 17)
start add(1, 9)
stop  add(7, 17)
stop  add(1, 9)
start add(11, 24)
stop  add(11, 24)
start add(10, 35)
stop  add(10, 35)


45

In [35]:
# Now we can apply our divide and conquer to other "reducer" tasks.

In [36]:
@task()
def mult(a, b):
    return a * b

In [37]:
scheduler.run(divide_and_conquer(mult, list(range(1, 10))))

362880

In [38]:
@task()
def pair(a, b):
    return (a, b)

In [39]:
scheduler.run(divide_and_conquer(pair, list(range(10))))

(((0, 1), (2, (3, 4))), ((5, 6), (7, (8, 9))))

In [40]:
# Tasks also support partial application.
inc = add.partial(1)
print(inc)
print(inc(10))
scheduler.run(inc(10))

PartialTask(fullname=redun_notebook.add, hash=6c7d07ac, args=(1,), kwargs={})
redun_notebook.add(1, 10)
start add(1, 10)
stop  add(1, 10)


11

In [41]:
# We can use task options to modify how tasks are run.
# For example, we can turn off caching on a per task-basis.
@task(cache=False)
def double(x):
    print("Doubling", x)
    return x * 2

In [42]:
# Notice we see the "Doubling" print statement occurs twice, so caching is disabled.
print(scheduler.run(double(1)))
print(scheduler.run(double(1)))

Doubling 1
2
Doubling 1
2


In [43]:
# We'll see in later examples, that when running tasks on a remote cluster, we can 
# specify options such as `memory`, `cpus`, and `gpus`.
# We can also override task options at call time.
# With caching, we won't see the "Doubling" print statements any more.
print(scheduler.run(double.options(cache=True)(1)))
print(scheduler.run(double.options(cache=True)(1)))

2
2


In [44]:
# Try your own expressions!